In [ ]:
from __future__ import print_function, division
%matplotlib notebook
import brainweb
from brainweb import volshow
import numpy as np
from os import path
from tqdm.auto import tqdm
import logging
logging.basicConfig(level=logging.INFO)
import nibabel as nib
import sirf.STIR as pet
import matplotlib.pyplot as plt
import os

In [ ]:
os.chdir(examples_data_path('PET'))
shutil.rmtree('working_folder/brainweb',True)
shutil.copytree('brain','working_folder/brainweb')
os.chdir('working_folder/brainweb')

In [ ]:
# Get the data. You'll need to run "pip install brainweb"
# if you don't have it already
file,url= list(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(file,url,".")
data = brainweb.load_file(file)

brainweb.seed(1337)

for f in tqdm([file], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1)
    vol_amyl = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1,
        PetClass=brainweb.Amyloid)

FDG  = vol['PET']
amyl = vol_amyl['PET']
uMap = vol['uMap']
T1   = vol['T1']
T2   = vol['T2']

In [ ]:
def subplot_(idx,vol,title,clims=None,cmap="viridis"):
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

plt.figure();
slice = FDG.shape[0]//2
subplot_([2,3,1],FDG [slice, 100:-100, 100:-100],'FDG'    ,cmap="hot")
subplot_([2,3,2],amyl[slice, 100:-100, 100:-100],'Amyloid',cmap="hot")
subplot_([2,3,3],uMap[slice, 100:-100, 100:-100],'uMap'   ,cmap="bone")
subplot_([2,3,4],T1  [slice, 100:-100, 100:-100],'T1'     ,cmap="Greys_r")
subplot_([2,3,5],T2  [slice, 100:-100, 100:-100],'T2'     ,cmap="Greys_r")

## Save as STIR interfile

In [ ]:
# We'll need a template sinogram
import sirf.STIR as pet
templ_sino = pet.AcquisitionData('mMR_template_span11.hs')

def crop_image(vol):
    return vol[:,17:17+285,17:17+285]

def save_as_interfile(templ_sino, vol, fname):
    # Crop to (127,285,285)
    vol = crop_image(vol)
    im = pet.ImageData(templ_sino)
    im.fill(vol)
    im.write(fname)
    # Use the zoom functionality
    small_fname = fname + "_small.hv"
    !zoom_image {small_fname} {fname}.hv 150 1 25 25
    # Remove all offset info
    !sed -i '' '/first pixel offset (mm)/d' {small_fname}
    # Read back in and return
    return pet.ImageData(small_fname)

FDG  = save_as_interfile(templ_sino, FDG,  "FDG"    )
amyl = save_as_interfile(templ_sino, amyl, "Amyloid")
uMap = save_as_interfile(templ_sino, uMap, "uMap"   )
T1   = save_as_interfile(templ_sino, T1,   "T1"     )
T2   = save_as_interfile(templ_sino, T2,   "T2"     )

## Forward project

In [ ]:
# We'll need a template sinogram
templ_sino = pet.AcquisitionData('mMR_template_span11.hs')

#%% create acquisition model
am = pet.AcquisitionModelUsingRayTracingMatrix()
am.set_num_tangential_LORs(5)

# Set up sensitivity due to attenuation
asm_attn = pet.AcquisitionSensitivityModel(uMap, am)
asm_attn.set_up(templ_sino)
bin_eff = pet.AcquisitionData(templ_sino)
bin_eff.fill(1.0)
print('applying attenuation (please wait, may take a while)...')
asm_attn.unnormalise(bin_eff)
asm_attn = pet.AcquisitionSensitivityModel(bin_eff)

am.set_acquisition_sensitivity(asm_attn)

am.set_up(templ_sino,uMap);

In [ ]:
# Function for adding noise
def add_noise(proj_data,noise_factor = 1):
    proj_data_arr = proj_data.as_array() / noise_factor
    noisy_proj_data_arr = np.random.poisson(proj_data_arr).astype('float32');
    noisy_proj_data = proj_data.clone()
    noisy_proj_data.fill(noisy_proj_data_arr);
    return noisy_proj_data

In [ ]:
# FDG
sino_FDG = am.forward(FDG)
sino_FDG.write("FDG_sino")
sino_FDG_noisy = add_noise(sino_FDG,1000)
sino_FDG_noisy.write("FDG_sino_noisy")

# Amyloid
sino_amyl = am.forward(amyl)
sino_amyl.write("amyl_sino")
sino_amyl_noisy = add_noise(sino_amyl,1000)
sino_amyl_noisy.write("amyl_sino_noisy")

In [ ]:
plt.figure();
subplot_([2,2,1],       sino_FDG.as_array()[0,60,:,:],'FDG'          )
subplot_([2,2,2], sino_FDG_noisy.as_array()[0,60,:,:],'Noisy FDG'    )
subplot_([2,2,3],      sino_amyl.as_array()[0,60,:,:],'Amyloid'      )
subplot_([2,2,4],sino_amyl_noisy.as_array()[0,60,:,:],'Noisy amyloid')